In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon, mapping, Point
import folium
from folium.plugins import HeatMap

# === 1. 광진구 행정구역 불러오기 및 필터 ===
shp_path = "Data/emd_20230729 (1)/emd.shp"
gdf_emd = gpd.read_file(shp_path, encoding='euc-kr')

# CRS 설정
if gdf_emd.crs is None:
    gdf_emd.set_crs(epsg=5179, inplace=True)

# 서울시, 광진구 필터링
seoul_gdf = gdf_emd[gdf_emd["EMD_CD"].astype(str).str.startswith("11")]
gwangjin_gdf = seoul_gdf[seoul_gdf["EMD_CD"].astype(str).str.startswith("11215")].copy()
gwangjin_gdf = gwangjin_gdf.to_crs(epsg=5179)

# === 2. 500m 격자 생성 ===
minx, miny, maxx, maxy = gwangjin_gdf.total_bounds
grid_size = 500
grid_list = []

x = minx
while x < maxx:
    y = miny
    while y < maxy:
        cell = Polygon([
            (x, y),
            (x + grid_size, y),
            (x + grid_size, y + grid_size),
            (x, y + grid_size)
        ])
        grid_list.append(cell)
        y += grid_size
    x += grid_size

# 격자 GeoDataFrame 생성 및 clip
grid_gdf = gpd.GeoDataFrame(geometry=grid_list, crs="EPSG:5179")
grid_clipped = gpd.clip(grid_gdf, gwangjin_gdf)

# 중심점 계산
centroid_gdf = gpd.GeoDataFrame(geometry=grid_clipped.geometry.centroid, crs="EPSG:5179")

# 지도 중심 계산
center = gwangjin_gdf.to_crs(epsg=4326).geometry.centroid.unary_union.centroid
map_center = [center.y, center.x]

# === 3. 지도 생성 ===
m = folium.Map(location=map_center, zoom_start=14, tiles="cartodbpositron")

# === 4. 광진구 경계 표시 ===
gwangjin_union = gwangjin_gdf.geometry.unary_union
folium.GeoJson(
    data=mapping(gwangjin_union),
    name="광진구 경계",
    style_function=lambda x: {"color": "black", "weight": 3, "fillOpacity": 0}
).add_to(m)

# === 5. 행정동 경계 ===
for geom in gwangjin_gdf.to_crs(epsg=4326).geometry:
    folium.GeoJson(
        data=mapping(geom),
        style_function=lambda x: {"color": "gray", "weight": 1, "fillOpacity": 0}
    ).add_to(m)

# === 6. 격자 표시 ===
for poly in grid_clipped.to_crs(epsg=4326).geometry:
    folium.GeoJson(
        data=mapping(poly),
        style_function=lambda x: {
            "fillColor": "#00000000",
            "color": "blue",
            "weight": 2,
            "opacity": 0.8,
        }
    ).add_to(m)

# === 7. 중심점 번호 라벨 ===
for idx, point in enumerate(centroid_gdf.to_crs(epsg=4326).geometry):
    folium.map.Marker(
        [point.y, point.x],
        icon=folium.DivIcon(
            icon_size=(150, 36),
            icon_anchor=(7, 7),
            html=f'<div style="font-size: 10pt; color: red;"><b>{idx+1}</b></div>'
        )
    ).add_to(m)

# === 8. 공원 위치 표시 ===
df = pd.read_csv('Data/전국도시공원정보표준데이터.csv', encoding='euc-kr')
pattern = "광진구"
filtered = df[
    df["소재지도로명주소"].astype(str).str.contains(pattern, na=False) |
    df["소재지지번주소"].astype(str).str.contains(pattern, na=False)
].copy()

if "위도" in filtered.columns and "경도" in filtered.columns:
    for _, row in filtered.iterrows():
        try:
            lat = float(row["위도"])
            lon = float(row["경도"])
            name = row.get("공원명", "이름 없음")
            folium.Marker(
                location=[lat, lon],
                popup=name,
                icon=folium.Icon(color="green", icon="tree-conifer", prefix="glyphicon")
            ).add_to(m)
        except:
            continue
else:
    print("❗ 위도/경도 컬럼이 없습니다. 주소 기반 좌표 변환 필요.")

# === 9. HeatMap 표시 ===
df_heat = pd.read_csv("Data/빌라데이타.csv")
df_heat["위도"] = pd.to_numeric(df_heat["위도"], errors="coerce")
df_heat["경도"] = pd.to_numeric(df_heat["경도"], errors="coerce")
df_heat_clean = df_heat.dropna(subset=["위도", "경도"])

heat_data = df_heat_clean[["위도", "경도"]].values.tolist()
heat_data = [[lat, lon] for lat, lon in heat_data if isinstance(lat, float) and isinstance(lon, float)]

heat_layer = HeatMap(
    heat_data,
    radius=5,
    blur=5,
    min_opacity=0.3,
    gradient={
        "0.2": "yellow",
        "0.4": "orange",
        "0.6": "red"
    }
)
heat_layer.add_to(m)

# === 10. 레이어 컨트롤 추가 및 출력 ===
folium.LayerControl().add_to(m)

# === 11. 광진구 어린이집 위치 표시 ===
df_kids = pd.read_csv("Data/광진구_실제운영중_어린이집.csv", encoding="utf-8")

# 위경도 컬럼 확인 및 변환
df_kids["위도"] = pd.to_numeric(df_kids["시설 위도(좌표값)"], errors="coerce")
df_kids["경도"] = pd.to_numeric(df_kids["시설 경도(좌표값)"], errors="coerce")

# NaN 제거
df_kids_clean = df_kids.dropna(subset=["위도", "경도"])

# 지도에 마커로 표시
for _, row in df_kids_clean.iterrows():
    try:
        lat = float(row["위도"])
        lon = float(row["경도"])
        name = row.get("어린이집명", "이름 없음")  # 컬럼명이 다를 경우 수정
        folium.Marker(
            location=[lat, lon],
            popup=f"👶 {name}",
            icon=folium.Icon(color="purple", icon="heart", prefix="fa")  # Font Awesome 아이콘 사용
        ).add_to(m)
    except:
        continue

# 결과 보기 (Jupyter 환경에서)
m

# 저장 옵션 (필요 시)
# m.save("final_gwangjin_map.html")

C:\Users\hsrhe\AppData\Local\Temp\ipykernel_29176\374552533.py:47: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = gwangjin_gdf.to_crs(epsg=4326).geometry.centroid.unary_union.centroid
C:\Users\hsrhe\AppData\Local\Temp\ipykernel_29176\374552533.py:47: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center = gwangjin_gdf.to_crs(epsg=4326).geometry.centroid.unary_union.centroid
C:\Users\hsrhe\AppData\Local\Temp\ipykernel_29176\374552533.py:54: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  gwangjin_union = gwangjin_gdf.geometry.unary_union


In [2]:
grid_count = grid_clipped.shape[0]
print(f"광진구 내부 500m 격자는 총 {grid_count}개입니다.")

광진구 내부 500m 격자는 총 93개입니다.


## 청크별 빌라 개수 시각화

In [3]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon, Point, mapping
import folium

# === 1. 광진구 행정구역 불러오기 ===
shp_path = "Data/emd_20230729 (1)/emd.shp"
gdf_emd = gpd.read_file(shp_path, encoding='euc-kr')

if gdf_emd.crs is None:
    gdf_emd.set_crs(epsg=5179, inplace=True)

# 광진구 필터
gwangjin_gdf = gdf_emd[gdf_emd["EMD_CD"].astype(str).str.startswith("11215")].copy()
gwangjin_gdf = gwangjin_gdf.to_crs(epsg=5179)

# === 2. 500m 격자 생성 ===
minx, miny, maxx, maxy = gwangjin_gdf.total_bounds
grid_size = 500
grid_list = []

x = minx
while x < maxx:
    y = miny
    while y < maxy:
        cell = Polygon([
            (x, y),
            (x + grid_size, y),
            (x + grid_size, y + grid_size),
            (x, y + grid_size)
        ])
        grid_list.append(cell)
        y += grid_size
    x += grid_size

grid_gdf = gpd.GeoDataFrame(geometry=grid_list, crs="EPSG:5179")
grid_clipped = gpd.clip(grid_gdf, gwangjin_gdf)

# === 3. 빌라 좌표 데이터 불러오기 ===
df_villa = pd.read_csv("Data/빌라데이타.csv")
df_villa["위도"] = pd.to_numeric(df_villa["위도"], errors="coerce")
df_villa["경도"] = pd.to_numeric(df_villa["경도"], errors="coerce")
df_villa = df_villa.dropna(subset=["위도", "경도"])

# 빌라 GeoDataFrame으로 변환 (좌표계: EPSG:4326 → EPSG:5179로 변환 필요)
gdf_villa = gpd.GeoDataFrame(
    df_villa,
    geometry=gpd.points_from_xy(df_villa["경도"], df_villa["위도"]),
    crs="EPSG:4326"
).to_crs(epsg=5179)

# === 4. 각 격자별로 빌라 수 세기 ===
grid_clipped["빌라수"] = grid_clipped.geometry.apply(
    lambda poly: gdf_villa.within(poly).sum()
)

# === 5. 지도 중심 계산 ===
center = gwangjin_gdf.to_crs(epsg=4326).geometry.centroid.unary_union.centroid
m = folium.Map(location=[center.y, center.x], zoom_start=14, tiles="cartodbpositron")

# === 6. 격자와 빌라 수 표시 ===
for idx, row in grid_clipped.iterrows():
    geom = row["geometry"]
    count = row["빌라수"]

    # ⭐ Polygon → GeoSeries → 위경도 변환
    geom_wgs = gpd.GeoSeries([geom], crs="EPSG:5179").to_crs(epsg=4326).iloc[0]

    # 격자 표시
    folium.GeoJson(
        data=mapping(geom_wgs),
        style_function=lambda x: {
            "fillColor": "#00000000",
            "color": "blue",
            "weight": 2,
            "opacity": 0.7,
        }
    ).add_to(m)

    # 중심 좌표 계산 (EPSG:5179 → 4326)
    centroid = geom.centroid
    centroid_wgs = gpd.GeoSeries([centroid], crs="EPSG:5179").to_crs(epsg=4326).iloc[0]

    # 빌라 수 텍스트 마커
    folium.map.Marker(
        location=[centroid_wgs.y, centroid_wgs.x],
        icon=folium.DivIcon(
            icon_size=(50, 20),
            icon_anchor=(10, 10),
            html=f'<div style="font-size: 9pt; color: red;"><b>{count}</b></div>'
        )
    ).add_to(m)

# === 7. 광진구 경계선 표시 ===
gwangjin_union = gwangjin_gdf.unary_union
gwangjin_union_wgs = gpd.GeoSeries([gwangjin_union], crs="EPSG:5179").to_crs(epsg=4326).iloc[0]

folium.GeoJson(
    data=mapping(gwangjin_union_wgs),
    name="광진구 경계",
    style_function=lambda x: {"color": "black", "weight": 3, "fillOpacity": 0}
).add_to(m)

# === 8. 개별 행정동 경계 표시 ===
for geom in gwangjin_gdf.geometry:
    geom_wgs = gpd.GeoSeries([geom], crs="EPSG:5179").to_crs(epsg=4326).iloc[0]
    folium.GeoJson(
        data=mapping(geom_wgs),
        style_function=lambda x: {"color": "gray", "weight": 5, "fillOpacity": 0}
    ).add_to(m)

# 결과 보기
m

C:\Users\hsrhe\AppData\Local\Temp\ipykernel_29176\2204491476.py:58: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = gwangjin_gdf.to_crs(epsg=4326).geometry.centroid.unary_union.centroid
C:\Users\hsrhe\AppData\Local\Temp\ipykernel_29176\2204491476.py:58: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center = gwangjin_gdf.to_crs(epsg=4326).geometry.centroid.unary_union.centroid
C:\Users\hsrhe\AppData\Local\Temp\ipykernel_29176\2204491476.py:95: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  gwangjin_union = gwangjin_gdf.unary_union


In [6]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon, box

# === 1. 광진구 행정구역 불러오기 ===
shp_path = "Data/emd_20230729 (1)/emd.shp"
gdf_emd = gpd.read_file(shp_path, encoding='euc-kr')

if gdf_emd.crs is None:
    gdf_emd.set_crs(epsg=5179, inplace=True)

# 광진구 필터: EMD_CD가 "11215"로 시작하는 행 (광진구 해당)
gwangjin_gdf = gdf_emd[gdf_emd["EMD_CD"].astype(str).str.startswith("11215")].copy()
gwangjin_gdf = gwangjin_gdf.to_crs(epsg=5179)

# === 2. 광진구 영역 내 500m 격자(청크) 생성 ===
minx, miny, maxx, maxy = gwangjin_gdf.total_bounds
grid_size = 500
grid_list = []
x = minx
while x < maxx:
    y = miny
    while y < maxy:
        cell = Polygon([
            (x, y),
            (x + grid_size, y),
            (x + grid_size, y + grid_size),
            (x, y + grid_size)
        ])
        grid_list.append(cell)
        y += grid_size
    x += grid_size

grid_gdf = gpd.GeoDataFrame(geometry=grid_list, crs="EPSG:5179")
# 광진구 영역으로 클립
grid_clipped = gpd.clip(grid_gdf, gwangjin_gdf)
grid_clipped = grid_clipped.reset_index(drop=True)
grid_clipped["청크ID"] = grid_clipped.index

# === 3. 빌라(주택) 데이터 불러오기 및 전처리 ===
df_villa = pd.read_csv("Data/빌라데이타.csv")
df_villa["위도"] = pd.to_numeric(df_villa["위도"], errors="coerce")
df_villa["경도"] = pd.to_numeric(df_villa["경도"], errors="coerce")
df_villa = df_villa.dropna(subset=["위도", "경도"])
df_villa["세대수"] = pd.to_numeric(df_villa["세대수"], errors="coerce").fillna(0)

# GeoDataFrame으로 변환 (좌표계 4326 → 5179)
gdf_villa = gpd.GeoDataFrame(
    df_villa,
    geometry=gpd.points_from_xy(df_villa["경도"], df_villa["위도"]),
    crs="EPSG:4326"
).to_crs(epsg=5179)

# === 4. 빌라 데이터와 500m 격자(청크) 매핑 ===
gdf_villa_chunk = gpd.sjoin(gdf_villa, grid_clipped[["청크ID", "geometry"]], how="left", predicate="within")
# 첫 번째 sjoin으로 추가된 'index_right' 열 제거
gdf_villa_chunk = gdf_villa_chunk.drop(columns=['index_right'], errors='ignore')

# === 5. 빌라 데이터와 행정동(동) 매핑 ===
# 광진구 행정동 정보 중 동 이름 컬럼은 "EMD_KOR_NM"로 가정
gdf_villa_full = gpd.sjoin(gdf_villa_chunk, gwangjin_gdf[["EMD_KOR_NM", "geometry"]], how="left", predicate="within")
# (참고: 두 번째 sjoin 후에 자동 생성된 'index_right' 열은 불필요하면 제거 가능)

# === 6. 청크별, 행정동별로 빌라 수와 세대수 집계 ===
villa_summary = gdf_villa_full.groupby(["청크ID", "EMD_KOR_NM"]).agg(
    빌라수=('geometry', 'count'),
    세대수=('세대수', 'sum')
).reset_index()

# === 7. 결과 저장 (CSV) ===
villa_summary.to_csv("Data\광진구_청크_동별_빌라_세대수.csv", index=False, encoding='utf-8-sig')

# 집계 결과 미리보기
print(villa_summary.head())


   청크ID EMD_KOR_NM  빌라수     세대수
0   5.0        자양동   87   498.0
1   6.0        자양동   88   361.0
2   7.0        자양동  615  1293.0
3   8.0        자양동  164  2323.0
4   9.0        자양동   13     0.0


In [7]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import mapping, Polygon
import folium

# === 1. 광진구 행정구역 불러오기 및 필터링 ===
shp_path = "Data/emd_20230729 (1)/emd.shp"
gdf_emd = gpd.read_file(shp_path, encoding='euc-kr')
if gdf_emd.crs is None:
    gdf_emd.set_crs(epsg=5179, inplace=True)
    
# EMD_CD가 "11215"로 시작하는 행(광진구에 해당)
gwangjin_gdf = gdf_emd[gdf_emd["EMD_CD"].astype(str).str.startswith("11215")].copy()
gwangjin_gdf = gwangjin_gdf.to_crs(epsg=5179)

# === 2. 광진구 영역 내 500m 격자(청크) 생성 ===
minx, miny, maxx, maxy = gwangjin_gdf.total_bounds
grid_size = 500
grid_list = []
x = minx
while x < maxx:
    y = miny
    while y < maxy:
        cell = Polygon([
            (x, y),
            (x + grid_size, y),
            (x + grid_size, y + grid_size),
            (x, y + grid_size)
        ])
        grid_list.append(cell)
        y += grid_size
    x += grid_size

grid_gdf = gpd.GeoDataFrame(geometry=grid_list, crs="EPSG:5179")
# 광진구 영역으로 클립
grid_clipped = gpd.clip(grid_gdf, gwangjin_gdf)
# 재인덱스 후 청크ID (원하는 경우 1부터 부여하려면: grid_clipped.index + 1)
grid_clipped = grid_clipped.reset_index(drop=True)
grid_clipped["청크ID"] = grid_clipped.index  # 여기서는 0부터 시작됨

# === 3. 빌라(주택) 데이터 불러오기 및 전처리 ===
df_villa = pd.read_csv("Data/빌라데이타.csv")
df_villa["위도"] = pd.to_numeric(df_villa["위도"], errors="coerce")
df_villa["경도"] = pd.to_numeric(df_villa["경도"], errors="coerce")
df_villa = df_villa.dropna(subset=["위도", "경도"])
df_villa["세대수"] = pd.to_numeric(df_villa["세대수"], errors="coerce").fillna(0)

# 4326 좌표계에서 GeoDataFrame 생성 후 5179로 변환
gdf_villa = gpd.GeoDataFrame(
    df_villa,
    geometry=gpd.points_from_xy(df_villa["경도"], df_villa["위도"]),
    crs="EPSG:4326"
).to_crs(epsg=5179)

# === 4. 빌라 데이터와 500m 격자(청크) 매핑 ===
gdf_villa_chunk = gpd.sjoin(gdf_villa, grid_clipped[["청크ID", "geometry"]], how="left", predicate="within")
# 첫 번째 sjoin 후에 생성된 'index_right' 제거
gdf_villa_chunk = gdf_villa_chunk.drop(columns=['index_right'], errors='ignore')

# (참고: 여기까지는 각 빌라에 해당 청크ID 할당)

# villa_summary는 이미 청크ID, EMD_KOR_NM, 빌라수, 세대수를 집계한 데이터입니다.
# 만약 기존 villa_summary가 없다면 아래와 같이 2중 공간 조인을 통해 행정동(동) 정보를 부여할 수 있습니다.
# (여기선 앞서 집계한 villa_summary가 있다고 가정하지 않고, 우선 전체 빌라의 세대수를 청크별로 집계합니다.)

# --- 청크별로 전체 빌라 수와 세대수 집계 ---
chunk_summary = gdf_villa_chunk.groupby("청크ID").agg(
    total빌라수=('geometry', 'count'),
    total세대수=('세대수', 'sum')
).reset_index()

# === 5. grid_clipped와 집계 데이터 병합 ===
grid_merged = grid_clipped.merge(chunk_summary, on="청크ID", how="left")
# 만약 해당 청크에 빌라가 하나도 없으면 NaN이 있으므로 0으로 채움
grid_merged["total세대수"] = grid_merged["total세대수"].fillna(0).astype(int)

# === 6. 지도 시각화를 위한 좌표계 변환 (EPSG:4326) ===
grid_merged_wgs = grid_merged.to_crs(epsg=4326)

# 광진구 중심 계산 (경계의 중심)
center = gwangjin_gdf.to_crs(epsg=4326).geometry.centroid.unary_union.centroid
m = folium.Map(location=[center.y, center.x], zoom_start=14, tiles="cartodbpositron")

# === 7. 각 청크(격자) 시각화 및 라벨 추가 ===
for idx, row in grid_merged_wgs.iterrows():
    geom = row["geometry"]
    chunk_id = row["청크ID"]
    total_households = row["total세대수"]
    
    # 격자 영역 GeoJson 추가 (윤곽선만 표시)
    folium.GeoJson(
        data=mapping(geom),
        style_function=lambda feature: {
            "fillColor": "#ffffff00",  # 투명 채우기
            "color": "blue",
            "weight": 2,
            "opacity": 0.7,
        },
    ).add_to(m)
    
    # 격자 중심 계산 및 좌표 변환
    centroid = geom.centroid
    centroid_wgs = gpd.GeoSeries([centroid], crs=grid_merged_wgs.crs).to_crs(epsg=4326).iloc[0]
    
    # 청크ID와 세대수 라벨 추가
    label_html = f'<div style="font-size:9pt; color:red; text-align:center;">청크 {chunk_id}<br>세대수 {total_households}</div>'
    folium.map.Marker(
        location=[centroid_wgs.y, centroid_wgs.x],
        icon=folium.DivIcon(icon_size=(50, 20), icon_anchor=(0, 0), html=label_html)
    ).add_to(m)

# 지도 출력
m


C:\Users\hsrhe\AppData\Local\Temp\ipykernel_29176\1767177581.py:81: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = gwangjin_gdf.to_crs(epsg=4326).geometry.centroid.unary_union.centroid
C:\Users\hsrhe\AppData\Local\Temp\ipykernel_29176\1767177581.py:81: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center = gwangjin_gdf.to_crs(epsg=4326).geometry.centroid.unary_union.centroid


### 빌라데이터 행정동 세분화

In [37]:
import os

def print_tree(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, "").count(os.sep)
        indent = "    " * level
        print(f"{indent}{os.path.basename(root)}/")
        for f in files:
            print(f"{indent}    {f}")

print_tree("Data")


Data/
    .DS_Store
    20250411_단지_면적정보.xlsx
    광진구_실제운영중_어린이집.csv
    광진구_청크_동별_빌라_세대수.csv
    빌라_세분화_법정동.csv
    빌라데이타.csv
    서울시 어린이집 정보(표준 데이터).csv
    서울특별시_광진구_도시공원정보_20250310.csv
    전국도시공원정보표준데이터.csv
    행정동별_인구비율_데이터.csv
    1. 통계/
        1. 2023년 행정구역 통계(인구)/
            2024년기준_2023년_성연령별인구.csv
            2024년기준_2023년_인구총괄(노년부양비).csv
            2024년기준_2023년_인구총괄(노령화지수).csv
            2024년기준_2023년_인구총괄(유년부양비).csv
            2024년기준_2023년_인구총괄(인구밀도).csv
            2024년기준_2023년_인구총괄(총인구).csv
            2024년기준_2023년_인구총괄(평균나이).csv
        2. 2023년 행정구역 통계(가구)/
            2024년기준_2023년_가구총괄.csv
            2024년기준_2023년_세대구성별가구.csv
        3. 2023년 행정구역 통계(주택)/
            2024년기준_2023년_건축년도별주택.csv
            2024년기준_2023년_연건평별주택.csv
            2024년기준_2023년_주택유형별주택.csv
            2024년기준_2023년_주택총괄_총주택(거처)수.csv
        4. 2022년 행정구역 통계(사업체, 종사자)/
            2024년기준_2022년_산업분류별(10차_대분류)_사업체수.csv
            2024년기준_2022년_산업분류별(10차_대분류)_종사자수.csv
            202

In [38]:
import os
import geopandas as gpd
import pandas as pd

# ──────────────────────────────────────────────
# [0] 환경 설정: 법정동(읍·면·동) 경계 shapefile 경로 확인
# ──────────────────────────────────────────────
shp_path = "Data/2. 경계/3. 2024년 2분기 기준 행정동 경계/bnd_dong_00_2024_2Q.shp"
if not os.path.exists(shp_path):
    raise FileNotFoundError(f"{shp_path} 파일이 존재하지 않습니다.")

# ──────────────────────────────────────────────
# [1] 빌라 데이터 불러오기
# ──────────────────────────────────────────────
df_villa = pd.read_csv("Data/빌라데이타.csv", encoding="utf-8")
df_villa = df_villa.dropna(subset=["위도", "경도"])
df_villa["위도"] = pd.to_numeric(df_villa["위도"], errors="coerce")
df_villa["경도"] = pd.to_numeric(df_villa["경도"], errors="coerce")

gdf_villa = gpd.GeoDataFrame(
    df_villa,
    geometry=gpd.points_from_xy(df_villa["경도"], df_villa["위도"]),
    crs="EPSG:4326"
)

# ──────────────────────────────────────────────
# [2] 행정동(법정동) 경계 shapefile 읽기
# ──────────────────────────────────────────────
gdf_emd = gpd.read_file(shp_path, encoding="euc-kr")

# (확인) 컬럼명 출력
print("EMD 레이어 컬럼:", gdf_emd.columns.tolist())
# 예시: ['BASE_DATE','ADM_NM','ADM_CD','geometry']

# 좌표계 지정(EPSG:5179) & WGS84(EPSG:4326) 변환
if gdf_emd.crs is None or gdf_emd.crs.to_epsg() != 5179:
    gdf_emd.set_crs(epsg=5179, inplace=True)
gdf_emd = gdf_emd.to_crs(epsg=4326)

# ──────────────────────────────────────────────
# [3] 광진구(ADM_CD '11215*') 법정동만 추출
# ──────────────────────────────────────────────
gdf_gj = gdf_emd[
    gdf_emd["ADM_CD"].astype(str).str.startswith("11215")
].copy()

# ──────────────────────────────────────────────
# [4] 공간 조인: 빌라 지점에 법정동명·코드 붙이기
# ──────────────────────────────────────────────
gdf_out = gpd.sjoin(
    gdf_villa,
    gdf_gj[["ADM_CD", "ADM_NM", "geometry"]],
    how="left",
    predicate="within"
).rename(columns={
    "ADM_CD": "법정동코드",
    "ADM_NM": "법정동명"
})

# ──────────────────────────────────────────────
# [5] 세대수 > 0 필터링
# ──────────────────────────────────────────────
gdf_out = gdf_out[gdf_out["세대수"] > 0].copy()

# ──────────────────────────────────────────────
# [6] 결과 저장 (geometry 컬럼 제외)
# ──────────────────────────────────────────────
cols = [c for c in gdf_out.columns if c != "geometry"]
gdf_out[cols].to_csv(
    "Data/빌라_법정동_세분화.csv",
    index=False,
    encoding="utf-8-sig"
)

# ──────────────────────────────────────────────
# [7] 결과 미리보기
# ──────────────────────────────────────────────
print(gdf_out[["대지위치", "법정동코드", "법정동명", "세대수"]].head(20))


c:\Users\hsrhe\anaconda3\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: One or several characters couldn't be converted correctly from euc-kr to UTF-8.  This warning will not be emitted anymore
  return ogr_read(


EMD 레이어 컬럼: ['BASE_DATE', 'ADM_NM', 'ADM_CD', 'geometry']
                     대지위치 법정동코드 법정동명   세대수
10    서울특별시 광진구 자양동 843-1   NaN  NaN  16.0
11  서울특별시 광진구 자양동 227-273   NaN  NaN  10.0
17   서울특별시 광진구 중곡동 257-21   NaN  NaN   3.0
18   서울특별시 광진구 중곡동 163-35   NaN  NaN   2.0
22    서울특별시 광진구 중곡동 29-22   NaN  NaN   6.0
23    서울특별시 광진구 중곡동 51-41   NaN  NaN   3.0
29    서울특별시 광진구 중곡동 103-6   NaN  NaN   6.0
30   서울특별시 광진구 중곡동 103-18   NaN  NaN   7.0
31   서울특별시 광진구 중곡동 104-20   NaN  NaN   7.0
32   서울특별시 광진구 중곡동 104-44   NaN  NaN   8.0
43   서울특별시 광진구 중곡동 647-11   NaN  NaN   6.0
44     서울특별시 광진구 능동 240-9   NaN  NaN   6.0
45    서울특별시 광진구 구의동 79-30   NaN  NaN   8.0
46   서울특별시 광진구 중곡동 141-19   NaN  NaN   8.0
47    서울특별시 광진구 중곡동 288-6   NaN  NaN   8.0
48    서울특별시 광진구 중곡동 294-9   NaN  NaN   8.0
49    서울특별시 광진구 중곡동 73-15   NaN  NaN   8.0
50   서울특별시 광진구 중곡동 101-20   NaN  NaN  16.0
52    서울특별시 광진구 중곡동 51-45   NaN  NaN   7.0
64    서울특별시 광진구 군자동 48-60   NaN  NaN   7.0


### 청크별 인구수 계산

In [9]:
import pandas as pd

# --- 1. 행정동별 인구비율 데이터 불러오기 ---
ratio_path = r"Data/행정동별_인구비율_데이터.csv"
# 인코딩은 상황에 맞게 'utf-8-sig' 또는 'cp949' 등을 사용하세요.
df_ratio = pd.read_csv(ratio_path, encoding='utf-8-sig')

# 행정동 컬럼명을 병합에 맞추기 위해 "행정동"을 "EMD_KOR_NM"으로 변경 (두 DataFrame의 행정동 기준 일치)
df_ratio = df_ratio.rename(columns={"행정동": "EMD_KOR_NM"})

# --- 2. 이전 단계에서 집계된 빌라 세대수 데이터 불러오기 (청크별, 동별) ---
villa_summary_path = r"Data/광진구_청크_동별_빌라_세대수.csv"
villa_summary = pd.read_csv(villa_summary_path, encoding='utf-8-sig')

# --- 3. 두 DataFrame 병합 (행정동 기준) ---
merged = pd.merge(villa_summary, df_ratio, on="EMD_KOR_NM", how="left")

# --- 4. 각 연령대 비율을 이용해 해당 연령대 세대수 산출 ---
# 비율 컬럼 리스트 (인구비율 데이터의 연령별 컬럼들)
age_columns = ["0~4세", "5~9세", "10~14세", "15~19세", "20~24세", "25~29세", "30~34세",
               "35~39세", "40~44세", "45~49세", "50~54세", "55~59세", "60~64세", "65~69세",
               "70~74세", "75~79세", "80~84세", "85~89세", "90~94세", "95~99세", "100세 이상"]

# 각 연령대의 추정 세대수는 (해당 청크의 세대수) x (해당 연령대 비율)
for col in age_columns:
    merged[f"{col}_세대수"] = merged["세대수"] * merged[col]

# --- 5. 필요한 컬럼만 선택하여 결과 DataFrame 구성 ---
cols_to_save = ["청크ID", "EMD_KOR_NM", "세대수"] + [f"{col}_세대수" for col in age_columns]
result_df = merged[cols_to_save]

# --- 6. 결과를 CSV 파일로 저장 ---
result_df.to_csv(r"Data/광진구_청크_동별_세대수_연령별추정.csv", index=False, encoding='utf-8-sig')

# 최종 결과 미리보기
print(result_df.head(20))


    청크ID EMD_KOR_NM     세대수  0~4세_세대수  5~9세_세대수  10~14세_세대수  15~19세_세대수  \
0    5.0        자양동   498.0       NaN       NaN         NaN         NaN   
1    6.0        자양동   361.0       NaN       NaN         NaN         NaN   
2    7.0        자양동  1293.0       NaN       NaN         NaN         NaN   
3    8.0        자양동  2323.0       NaN       NaN         NaN         NaN   
4    9.0        자양동     0.0       NaN       NaN         NaN         NaN   
5   13.0        자양동  2351.0       NaN       NaN         NaN         NaN   
6   14.0        자양동  1249.0       NaN       NaN         NaN         NaN   
7   15.0        자양동   844.0       NaN       NaN         NaN         NaN   
8   16.0        자양동  2362.0       NaN       NaN         NaN         NaN   
9   17.0        구의동    66.0       NaN       NaN         NaN         NaN   
10  17.0        자양동   573.0       NaN       NaN         NaN         NaN   
11  18.0        자양동  1440.0       NaN       NaN         NaN         NaN   
12  19.0        구의동  1147